# Tensor Network Renormalization Group for the 2D Ising model

In [1]:
using LinearAlgebra
using Plots

Define the basic functions for the Ising model

Define Kronecker Delta

In [2]:
function kronecker(i::Int, j::Int)::Int
    if i == j
        return 1
    else
        return 0 
    end
end

kronecker (generic function with 1 method)

In [3]:
kronecker(1, 1), kronecker(1, 0)

(1, 0)

Define the weight functions of Ising model

$Z_2$ has two representations, $k=0$ and $k=1$. The weight on an edge is $\cosh(\beta)$ for $k=0$, $\sinh(\beta)$ for $k=1$.

In [4]:
function weight(beta::Float64, k::Int)::Float64
    (1 - k) * cosh(beta) + k * sinh(beta)
end

weight (generic function with 1 method)

In [5]:
weight(0.0, 0), weight(0.0, 1)

(1.0, 0.0)

Define the initial tensor

The tensor is 4-valent, each edge carrying a representation of $Z_2$, and a square root of an edge weight (for $\beta$ and $k$). On the vertex, we have a Kronecker Delta of the sum of the 4 representations meeting at the vertex modulo 2; if this sum is 1, it vanishes.

Note that Julia starts counting array indices with 1, not 0. You need to account for that.

In [8]:
function tensor(beta, m = 2)
    array = zeros(m, m, m, m)
    
    for i in 1:m, j in 1:m, k in 1:m, l in 1:m
        mask = kronecker((i + j + k + l) % m, 0)
        array[i, j, k, l] = mask * sqrt(weight(beta, i - 1) * weight(beta, j - 1) * 
                                        weight(beta, k - 1) * weight(beta, l - 1))
    end
    array
end

tensor (generic function with 2 methods)

In [9]:
t = tensor(1.0)

2×2×2×2 Array{Float64,4}:
[:, :, 1, 1] =
 2.3811  0.0    
 0.0     1.81343

[:, :, 2, 1] =
 0.0      1.81343
 1.81343  0.0    

[:, :, 1, 2] =
 0.0      1.81343
 1.81343  0.0    

[:, :, 2, 2] =
 1.81343  0.0   
 0.0      1.3811

In [11]:
length(t[:,1,1,1])

2

Splitting tensors for the singular value decomposition

This will be similar to the introductory Julia File. We pair the indices of the tensor in the way we want to split it, and define a matrix. On this matrix we will apply a singular value decomposition, truncate here at 2 singular values, and define new 3-valent tensors from it.

Translating back just means splitting the combined index again into two separate ones.

The function should return two 3-valent tensors and a list of singular values.

Normalize the singular values with respect to the largest one (so the largest singular value is always one).

We need to the same again, but this time for the other splitting.

To get the new effective tensor, we now have to contract the fine degrees of freedom.

To compute the flow of renormalized tensors, we have to iterate these procedures a few times. We write this into a main function.

We have to specify the number of iterations of the algorithm, then define the initial tensor. We can pass $\beta$ as a parameter of the main function.

Then we do a for loop, in which we do the first splitting, second splitting and then contract the new three valent tensor. Then we repeat.

For each iteration we should store the singular values computed in the singular value decomposition. We will afterwards plot them to distinguish the phases of the Ising model and finding the phase transition.

If it is running, try the algorithm for different initial values of $\beta$ and look for a phase transition. The actual critical inverse temperature of the 2D square lattice Ising model is $\beta_c \approx 0.4406...$. The plot the singular values for different betas.

Plot the singular values, actually the second largest one. The x-axis is simply the iteration number.